In [1]:
from gurobipy import *  # import the optimize solver Gurobi

In [2]:
horizon_length = 3 # Set the index t for number of periods(1, 2, 3)
number_of_product = 2 # Set the index i for number of products(Street bike, road bike)
number_of_line = 2 # Set the index j for number of line(1, 2)
number_of_resources = 2 # Set the index k for number of resource(machines, workers)
m = Model() # Import and create the model

Using license file C:\Users\chaoz\gurobi_licence\gurobi.lic
Academic license - for non-commercial use only


In [3]:
# Set the input Parameter: 
forecasted_demand = [[1000,500],[1050,600],[1100,550]]# forecasted number of units demanded for product i in period t
mach_resource_lim = [8600,8500,8800] # amount of resource machine available in period t
worker_resource_lim = [17000, 16600, 17200] # amount of resource worker available in period t
mach_unit_resource_req = [[6,5],[7,6]] # amount of resources machine required by line j in one unit of product i
worker_unit_resource_req = [[4,10],[5,12]] # amount of resources worker required by line j in one unit of product i
line_cost = [[[72,80],[74,78],[75,78]],[[85,90],[88,95],[84,92]]]
holding_cost = [[5,6],[6,7],[5,7]]  #  cost to hold one unit of product i in inventory for period 
init_inv = [100,50] # initial inventory of product i

In [4]:
# Set the Variable nx : Number of units of bike i produced by line j in period t
# Set var nx to integer number

nx = []
for i in range(number_of_product):
    s = []
    for j in range(number_of_line):
        k = []
        for t in range(horizon_length):
            k.append(m.addVar(vtype=GRB.INTEGER, name='nx{}'.format(str(i + 1) + str(j + 1) + str(t + 1))))
        s.append(k)
    nx.append(s)
    

In [5]:
#Caculate number of units of product i held in inventory at the end of period t


I = []
for i in range(number_of_product):
    Is = []
    for t in range(horizon_length):
        if t == 0:
            iv = init_inv[i]
        else:
            iv = Is[t-1]
        pro = 0
        for j in range(number_of_line):
            pro += nx[i][j][t]
        fv = iv + pro - forecasted_demand[t][i]
        Is.append(fv)
    I.append(Is)
   
              

In [6]:
# Set the Minimize Obijective: Total Cost

m.setObjective(quicksum([line_cost[i][t][j] * nx[i][j][t] for j, t, i in [(x, y, z ) for x in range(number_of_line) for y in range (horizon_length) for z in range(number_of_product)]]) 
               + quicksum([holding_cost[t][i] * I[i][t] for i, t in[(w,v) for w in range(number_of_product) for v in range (horizon_length)]])
               ,GRB.MINIMIZE)

In [7]:
# Set Non Negative number of units of product

for i in range(number_of_product):
     for j in range(number_of_line):
            for t in range(horizon_length):
                c1 = m.addConstr(nx[i][j][t]  >= 0)
            

In [8]:
# Set Non Negative number of end of inventory

for i in range(number_of_product):
    for t in range(horizon_length):
        c2 = m.addConstr(I[i][t]  >= 0)

In [9]:
# Satisfy available factory capacity
# Worker capacity

c3 = []
for t in range(horizon_length):
    c3.append(m.addConstr(quicksum([nx[i][j][t]* worker_unit_resource_req[i][j] for i, j in [(a,b) for a in range(number_of_product) for b in range(number_of_line)]]) <= worker_resource_lim[t]))

In [10]:
# Satisfy available factory capacity
# Machine capacity

c4 = []
for t in range(horizon_length):
    c4.append(m.addConstr(quicksum([nx[i][j][t]* mach_unit_resource_req[i][j] for i, j in [(a,b) for a in range(number_of_product) for b in range(number_of_line)]]) <= mach_resource_lim[t]))

In [11]:
# Run the optimize solver
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 24 rows, 12 columns and 60 nonzeros
Model fingerprint: 0x0db6525b
Variable types: 0 continuous, 12 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [8e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 2e+04]
Presolve removed 12 rows and 0 columns
Presolve time: 0.01s
Presolved: 12 rows, 12 columns, 48 nonzeros
Variable types: 0 continuous, 12 integer (0 binary)

Root relaxation: objective 3.796422e+05, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 379642.222    0    7          - 379642.222      -     -    0s
H    0     0                    379663.00000 379642.222  0.01%     -    0s

Explored 1 nodes (15 simplex iterations) in 0.08 seconds
Thread count was 8 (of 8 available processors)

Solution co

In [12]:
# Get the Optimal Solution for X
m.printAttr('X')


    Variable            X 
-------------------------
       nx111          897 
       nx112            1 
       nx113            1 
       nx121            4 
       nx122         1056 
       nx123         1091 
       nx212          106 
       nx213           45 
       nx221          533 
       nx222          412 
       nx223          504 


In [13]:
# Get the Optimal Objective Value
m.ObjVal

379663.0